# Experimento de extração de arquivos PDF de repositórios digitais

Usa arquivo de resultados exportado pela biblioteca nacional de teses e dissertações

Consulta feita no dia xx/09/2024, limitando a documentos a partir de 2021 e com as mesmas palavras-chave do artigo da Regis.



In [6]:
# funções usadas no restante do código

import requests, re
import csv, ast, time, pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

################################################################################

def baixar_pdf(url, nome_arquivo):
    """
    Baixa um PDF de uma URL e salva em um arquivo local.

    Args:
        url: A URL do PDF.
        nome_arquivo: O nome do arquivo local onde o PDF será salvo.
    """

    resposta = requests.get(url)
    resposta.raise_for_status()  # Lança uma exceção se houver um erro na requisição

    with open(nome_arquivo, 'wb') as arquivo:
        arquivo.write(resposta.content)

    print(f"PDF baixado e salvo em: {nome_arquivo}")

################################################################################    

# Função para extrair URLs de uma string usando expressão regular
def extrair_url(texto):
  urls = re.findall(r'https?://[^\s]+', texto)
  return urls

def encontrar_links_pdf(url):
  """
  Encontra links para arquivos PDF em uma página web.

  Args:
    url: A URL da página web.

  Returns:
    Uma lista de links para arquivos PDF.
  """

  response = requests.get(url, verify=False)
  soup = BeautifulSoup(response.content, 'html.parser')

  links_pdf = []

  # Encontra todas as tags <a>
  for link in soup.find_all('a'):
    href = link.get('href')
    if href:
      # Verifica se o link termina com ".pdf" ou se a URL contém ".pdf"
      # alternativamente, se não encontrar, localizar o primeiro com 'bitstream' no meio e download no final
      if href.endswith('.pdf') or re.search(r'\.pdf', href):
        links_pdf.append(href)

  return links_pdf


################################################################################

In [4]:
# Programa principal

# carrega planilha com resultados gerados pelo catálogo digital

arquivo = "Resultados.xlsx"
dados = pandas.read_excel(arquivo)

dados.head()

,Autor(a),ID Lattes do(a) autor(a),Orientadores,ID Lattes dos orientadores,Membros da banca,ID Lattes dos membros da banca,Título,Ano de defesa,Instituição de defesa,Sigla da instituição de defesa,...,Área do conhecimento CNPq,Tipos de acesso,Tipo de documento,Assuntos em português,Assuntos em inglês,Idioma,Resumo,Link de acesso,Resumo em Inglês,Referência Bibliográfica
0,"Alzate Rubio, Valentina",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,CO2 mineralization process in basalt rocks: st...,2024,Universidade de São Paulo (USP),USP,...,Não informado pela instituição,openAccess,masterThesis,Basalt's petrography||Carbon capture and geolo...,Não informado pela instituição,eng,"The Serra Geral Formation basalts, prominent w...",https://www.teses.usp.br/teses/disponiveis/44/...,Não informado pela instituição,Não informado pela instituição
1,"Cabrera Ruiz, Maria Liceth, 1995-",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Relação entre as propriedades geomecânicas e o...,2022,Universidade Estadual de Campinas (UNICAMP),UNICAMP,...,Não informado pela instituição,openAccess,masterThesis,Geomecânica||Fácies (Geologia)||Petrofísica||P...,Não informado pela instituição,por,Orientador: Alessandro Batezelli,https://hdl.handle.net/20.500.12733/6331||CABR...,Não informado pela instituição,Não informado pela instituição
2,"Rojas Caro, Daniel Mauricio, 1996-",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Modelo geomecânico dos reservatórios do pré-sa...,2022,Universidade Estadual de Campinas (UNICAMP),UNICAMP,...,Não informado pela instituição,openAccess,masterThesis,Geomecânica||Carbonatos||Pré-sal||Falhas (Geol...,Não informado pela instituição,por,Orientador: Alessandro Batezelli,https://hdl.handle.net/20.500.12733/7666||ROJA...,Não informado pela instituição,Não informado pela instituição
3,"Cruz, Carolina Barbosa Leite da",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Luminescence signals sensitivity of quartz fro...,2023,Universidade de São Paulo (USP),USP,...,Não informado pela instituição,openAccess,masterThesis,Andes orientais||Bacia do Putumayo||Eastern An...,Não informado pela instituição,eng,Sediment provenance analysis is a widely used ...,https://www.teses.usp.br/teses/disponiveis/44/...,Não informado pela instituição,Não informado pela instituição
4,"Barbosa, Theresa Rocco Pereira",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição,The role of airborne geophysical data as covar...,2023,Universidade Federal de Santa Maria (UFSM),UFSM,...,Não informado pela instituição,openAccess,masterThesis,Gamaespectrometria||Magnetometria||Dados legad...,Não informado pela instituição,por,A porção sul da Bacia Sedimentar do Paraná tem...,http://repositorio.ufsm.br/handle/1/31268,Não informado pela instituição,Não informado pela instituição


In [5]:
# filtra por registros que sejam de acesso aberto e cria dataframe com somente algumas colunas relevantes
dados_acesso_aberto = dados[dados['Tipos de acesso']=="openAccess"]
dados_relevantes = dados_acesso_aberto[['Ano de defesa', 'Sigla da instituição de defesa', 'Tipo de documento', 'Link de acesso']]

# agrupa por instituição (visto que o tipo de informação que cada uma apresenta é diferente)
dados_by_instituicao = dados_relevantes.groupby('Sigla da instituição de defesa')

# para cada grupo, mostra na tela a URL do trabalho na biblioteca original
for nome_grupo, dados_grupo in dados_by_instituicao:
    print(f'Grupo: {nome_grupo}')
    
    for i, reg in dados_grupo.iterrows():
        #TODO: algumas contém mais do que uma URL. Identificar qual usar!!!
        print(f"  URL: {extrair_url(reg['Link de acesso'])}") 

    print('---')

Grupo: CPRM
  URL: ['https://rigeo.sgb.gov.br/handle/doc/24119']
  URL: ['https://rigeo.cprm.gov.br/handle/doc/22892']
  URL: ['https://rigeo.cprm.gov.br/handle/doc/22626']
  URL: ['https://rigeo.cprm.gov.br/handle/doc/22659']
  URL: ['https://rigeo.sgb.gov.br/handle/doc/24629']
  URL: ['https://rigeo.cprm.gov.br/handle/doc/23433']
---
Grupo: CUB
  URL: ['https://repositorio.cruzeirodosul.edu.br/handle/123456789/3828']
---
Grupo: FEI
  URL: ['https://doi.org/10.31414/EM.2022.D.131393.||https://doi.org/10.31414/EM.2022.D.131393||https://repositorio.fei.edu.br/handle/FEI/4411']
  URL: ['https://doi.org/10.31414/EQ.2022.D.131605.||https://doi.org/10.31414/EQ.2022.D.131605||https://repositorio.fei.edu.br/handle/FEI/4737']
  URL: ['https://doi.org/10.31414/EM.2023.D.131632.||https://doi.org/10.31414/EM.2023.D.131632||https://repositorio.fei.edu.br/handle/FEI/4861']
  URL: ['https://doi.org/10.31414/EQ.2021.D.131349.||https://doi.org/10.31414/EQ.2021.D.131349||https://repositorio.fei.edu.br/

In [8]:
# teste para recuperar somente os dados de um dos grupos
primeiro_grupo = dados_by_instituicao.get_group(list(dados_by_instituicao.groups.keys())[0])
primeiro_grupo

,Ano de defesa,Sigla da instituição de defesa,Tipo de documento,Link de acesso
30,2023,CPRM,doctoralThesis,https://rigeo.sgb.gov.br/handle/doc/24119
45,2022,CPRM,masterThesis,https://rigeo.cprm.gov.br/handle/doc/22892
55,2021,CPRM,masterThesis,https://rigeo.cprm.gov.br/handle/doc/22626
56,2021,CPRM,masterThesis,https://rigeo.cprm.gov.br/handle/doc/22659
79,2023,CPRM,masterThesis,https://rigeo.sgb.gov.br/handle/doc/24629
147,2022,CPRM,doctoralThesis,https://rigeo.cprm.gov.br/handle/doc/23433


In [22]:
# teste para recuperar a url do primeiro registro do primeiro grupo
list(primeiro_grupo['Link de acesso'])[0]

'https://rigeo.sgb.gov.br/handle/doc/24119'

In [9]:
# Analisa a página do primeiro elemento do primeiro grupo, localiza as URLs dos PDFs e mostra na tela 

url = list(primeiro_grupo['Link de acesso'])[0] # Substitua pela URL desejada
links = encontrar_links_pdf(url)  
parsed_url = urlparse(url)
servidor = parsed_url.netloc
for link in links:
  print(f"https://{servidor}{link}")

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rigeo.sgb.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://rigeo.sgb.gov.br/bitstream/doc/24119/1/tese__diener_fernando.pdf
https://rigeo.sgb.gov.br/bitstream/doc/24119/1/tese__diener_fernando.pdf


In [10]:
# arrumar (com o de cima) pois está pegando o campo url que pode conter textos (e não só urls)
# cuidar porque pode haver mais de uma
for nome_grupo, dados_grupo in dados_by_instituicao:
  print(f'Processando dados da: {nome_grupo}')
  print(dados_grupo)        
  
  url = list(dados_grupo['Link de acesso'])[0] 
  
  links = encontrar_links_pdf(url)  
  parsed_url = urlparse(url)
  servidor = parsed_url.netloc

  for link in links:
      print(f"https://{servidor}{link}")

  print('---')  

Processando dados da: CPRM
     Ano de defesa Sigla da instituição de defesa Tipo de documento  \
30            2023                           CPRM    doctoralThesis   
45            2022                           CPRM      masterThesis   
55            2021                           CPRM      masterThesis   
56            2021                           CPRM      masterThesis   
79            2023                           CPRM      masterThesis   
147           2022                           CPRM    doctoralThesis   

                                 Link de acesso  
30    https://rigeo.sgb.gov.br/handle/doc/24119  
45   https://rigeo.cprm.gov.br/handle/doc/22892  
55   https://rigeo.cprm.gov.br/handle/doc/22626  
56   https://rigeo.cprm.gov.br/handle/doc/22659  
79    https://rigeo.sgb.gov.br/handle/doc/24629  
147  https://rigeo.cprm.gov.br/handle/doc/23433  
https://rigeo.sgb.gov.br/bitstream/doc/24119/1/tese__diener_fernando.pdf
https://rigeo.sgb.gov.br/bitstream/doc/24119/1/tese_

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rigeo.sgb.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


---
Processando dados da: FEI
     Ano de defesa Sigla da instituição de defesa Tipo de documento  \
228           2022                            FEI      masterThesis   
409           2022                            FEI      masterThesis   
541           2023                            FEI      masterThesis   
735           2021                            FEI      masterThesis   

                                        Link de acesso  
228  LOPES, J. G. <b> Análise exergética de um sist...  
409  NEVES, Mayara Alves Rosa. <b> Propriedades int...  
541  CASTRO, João Vítor Mateika de. <b> Propriedade...  
735  LUIS, Daniela Denleschi Fernandes. <b> Modelag...  


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.cruzeirodosul.edu.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


InvalidSchema: No connection adapters were found for 'LOPES, J. G. <b> Análise exergética de um sistema de compressão de CO2 de uma plataforma FPSO. </b> 2022. 146 f. Dissertação (Mestrado em Engenharia Mecânica) - Centro Universitário FEI, São Bernardo do campo, 2022 Disponível em: https://doi.org/10.31414/EM.2022.D.131393.||https://doi.org/10.31414/EM.2022.D.131393||https://repositorio.fei.edu.br/handle/FEI/4411'

In [ ]:
# baixa os pdfs (ajustar pois não funciona ainda)
for recurso in dados_do_csv:
    identificador = ast.literal_eval(recurso['identifier'])

    pdf_sub_url = f"{identificador[0].removeprefix("http://hdl.handle.net")}/handle/{identificador[1]}"
    url_sufix = ".pdf?sequence=1&isAllowed=y"
    full_url = f"{server_url}{pdf_sub_url}{url_sufix}"

    print(f"Baixando PDF: {full_url}")

    nome_do_arquivo = f"{dados_do_csv[0]['type']}".removeprefix("['").removesuffix("']")
    nome_do_arquivo = f"{ast.literal_eval(dados_do_csv[0]['date'])[1]}_{nome_do_arquivo}_{identificador[1]}.pdf"

    baixar_pdf(full_url, nome_do_arquivo)

    time.sleep(1)
